In [22]:
import requests
import matplotlib.pyplot as plt
from IPython.display import Image
import pandas as pd
import json
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from shapely.geometry import Point
import numpy as np

### Establishing endpoint and parameters, submit api request, and converting to json

In [23]:
endpoint = "https://data.nashville.gov/resource/2u6v-ujjs.json?$limit=50000"

params = {'$where' : "incident_reported between '2021-01-13T00:00:00.000' and '2021-06-30T00:00:00.000'", 
}

In [24]:
response = requests.get(endpoint, params)

In [25]:
res = response.json()

### Converting response to a dataframe and cleanup

In [41]:
#converts response to dataframe with desired columns
res_trimmed = (pd.DataFrame(res, columns=['incident_reported', 
                                          'incident_number',
                                          'offense_description', 
                                          'latitude', 
                                          'longitude']))

#fixes incident reports to include only date in YYYY-MM-DD format
res_trimmed['incident_reported'] = (res_trimmed['incident_reported']
                                    .str.split("T", expand=True)[0])
print(res_trimmed.shape)
#trims dataframe to only contain offense_description of burglary
burglary_reports = (res_trimmed[res_trimmed['offense_description']
                                .str.contains('BURGLARY- AGGRAVATED')])
print(burglary_reports.shape)
burglary_reports.drop_duplicates(subset='incident_number', inplace=True)

print(burglary_reports.shape)

(48921, 5)
(984, 5)
(775, 5)


In [42]:
#converts lat & long columns from objects to floats
burglary_reports[['latitude', 'longitude']] = (burglary_reports[['latitude', 'longitude']]
                                               .apply(pd.to_numeric))


C:\Users\Conrad\anaconda3\envs\geospatial\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [43]:
burglary_reports.to_csv("../geospatial_api-silver-sharks/data/burglary_reports.csv")
